## Tutorial: Displaying gym Game in Jupyter Notebook

ref: http://mckinziebrandon.me/TensorflowNotebooks/2016/12/21/openai.html



In [20]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

In [2]:
def display_frames_as_gif(frames):
    """
    Displays a list of frames as a git, with controls
    """
    patch = plt.imshow(frames[0])
    plt.axis('off')
    
    def animate(i):
        patch.set_data(frames[i])
    
    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)
    display(display_animation(anim, default_mode='loop'))

In [8]:
env = gym.make('CartPole-v0')

observation = env.reset()
cum_reward = 0
frames = []
for t in range(5000):
    frames.append(env.render(mode = 'rgb_array'))
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    print(observation, reward, done, info)
    if done:
        print("Finished at step %d" % t)
        break
env.render(close=True)
# display_frames_as_gif(frames)

[2017-11-13 23:41:49,603] Making new env: CartPole-v0


[ 0.00233986 -0.19453108  0.02935758  0.2976697 ] 1.0 False {}
[-0.00155076  0.00016034  0.03531097  0.01438829] 1.0 False {}
[-0.00154755 -0.19544975  0.03559874  0.3179998 ] 1.0 False {}
[-0.00545655 -0.39106017  0.04195874  0.62169334] 1.0 False {}
[-0.01327775 -0.58674215  0.0543926   0.92729023] 1.0 False {}
[-0.02501259 -0.78255468  0.07293841  1.23655818] 1.0 False {}
[-0.04066369 -0.9785341   0.09766957  1.55117059] 1.0 False {}
[-0.06023437 -0.78470994  0.12869298  1.29048917] 1.0 False {}
[-0.07592857 -0.98121189  0.15450277  1.62053602] 1.0 False {}
[-0.0955528  -0.7882108   0.18691349  1.37972618] 1.0 False {}
[-0.11131702 -0.59584838  0.21450801  1.15084143] 1.0 True {}
Finished at step 10


## Training a Q Learning Network

ref: https://keon.io/deep-q-learning/

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [25]:
import random
from collections import deque

class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=200)
        self.gamma = 0.95  # discount rate 
        self.epsilon = 1.0 # exploration rate 
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * 
                          np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def load(self, name):
        self.model.load_weights(name)
    
    def save(self, name):
        self.model.save_weights(name)

In [29]:
env = gym.make('CartPole-v1')
state_size = env.observation_space.shape[0]  # 4
action_size = env.action_space.n  # 2
agent = DQNAgent(state_size, action_size)
done = False
batch_size = 32
episodes = 10000

for e in range(1, episodes+1):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done and e % 1000 == 0:
            print("episode: {}/{}, score: {}, e: {:.2f}".format(
                e, episodes, time, agent.epsilon))
        if done:
            break
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)

[2017-11-14 00:01:25,851] Making new env: CartPole-v1


episode: 1000/10000, score: 139, e: 0.01
episode: 2000/10000, score: 201, e: 0.01
episode: 3000/10000, score: 154, e: 0.01
episode: 4000/10000, score: 131, e: 0.01
episode: 5000/10000, score: 193, e: 0.01
episode: 6000/10000, score: 206, e: 0.01
episode: 7000/10000, score: 163, e: 0.01
episode: 8000/10000, score: 109, e: 0.01
episode: 9000/10000, score: 114, e: 0.01
episode: 10000/10000, score: 119, e: 0.01
